Read the input

In [140]:
using System.IO;
var input = await File.ReadAllLinesAsync("input.txt");


Define data types. The `Instruction` interface has an `Apply` method which transforms a `StringBuilder` according to the instructions.

Starting with the password to be scrambled, apply each operation in succession to the string. The individual operations behave as follows:

* swap position X with position Y means that the letters at indexes X and Y (counting from 0) should be swapped.
* swap letter X with letter Y means that the letters X and Y should be swapped (regardless of where they appear in the string).
* rotate left/right X steps means that the whole string should be rotated; for example, one right rotation would turn abcd into dabc.
* rotate based on position of letter X means that the whole string should be rotated to the right based on the index of letter X (counting from 0) as determined before this instruction does any rotations. Once the index is determined, rotate the string to the right one time, plus a number of times equal to that index, plus one additional time if the index was at least 4.
* reverse positions X through Y means that the span of letters at indexes X through Y (including the letters at X and Y) should be reversed in order.
* move position X to position Y means that the letter which is at index X should be removed from the string, then inserted such that it ends up at index Y.

In [171]:


interface Instruction 
{
    StringBuilder Apply(StringBuilder input);
    Instruction Reverse();
}

// move position X to position Y means that the letter which is at index X should be removed 
// from the string, then inserted such that it ends up at index Y.
record struct Move(int from, int to) : Instruction
{
    public StringBuilder Apply(StringBuilder input) 
    {
        var c = input[from];
        return input.Remove(from, 1).Insert(to, c);
    }
    public Instruction Reverse() => new Move(to, from);
}

record struct SwapP(int x, int y): Instruction
{
    public StringBuilder Apply(StringBuilder input)
    {
        var c = input[x];
        input[x] = input[y];
        input[y] = c;
        return input;
    }
    public Instruction Reverse() => this;
}

record struct SwapL(char a, char b): Instruction
{
    public StringBuilder Apply(StringBuilder input) 
    {
        var (x,y) = (-1, -1);
        for (int i = 0; i < input.Length; i++)
        {
            if (input[i] == a) x = i;
            if (input[i] == b) y = i;
        }
        return new SwapP(x,y).Apply(input);
    }
    public Instruction Reverse() => this;
}

record struct Reverse(int x, int y): Instruction
{
    public StringBuilder Apply(StringBuilder input) 
    {
        var sb = new StringBuilder();
        for (int i = y; i >= x; i--)
        {
            sb.Append(input[i]);
        }
        input.Remove(x, sb.Length);
        input.Insert(x, sb);
        return input;
    }
    Instruction Instruction.Reverse() => this;
}

// rotate left/right X steps means that the whole string should be rotated; for example, one right rotation would turn abcd into dabc.
record struct RotateL(int n): Instruction
{
    public StringBuilder Apply(StringBuilder input) 
    {
        for (int i = 0; i < n; i++)
        {
            input.Append(input[0]);
            input.Remove(0, 1);
        }
        return input;
    }
    public Instruction Reverse() => new RotateR(n);
}

// rotate left/right X steps means that the whole string should be rotated; for example, one right rotation would turn abcd into dabc.
record struct RotateR(int n): Instruction
{
    public StringBuilder Apply(StringBuilder input) 
    {
        for (int i = 0; i < n; i++)
        {
            input.Insert(0, input[input.Length-1]);
            input.Remove(input.Length - 1, 1);
        }
        return input;
    }
    public Instruction Reverse() => new RotateL(n);
}


// rotate based on position of letter X means that the whole string should be rotated to the right based on the 
// index of letter X (counting from 0) as determined before this instruction does any rotations. Once the index 
// is determined, rotate the string to the right one time, plus a number of times equal to that index, plus 
// one additional time if the index was at least 4.
record struct Rotate(char l): Instruction
{
   


    public StringBuilder Apply(StringBuilder input) 
    {
        int index = -1;
        for (int i = 0; i < input.Length; i++)
        {
            if (input[i] == l)
            {
                index = i;
                break;
            }
        }

        var rotations = index + 1;
        if (index >= 4) rotations ++;

        return new RotateR(rotations).Apply(input);
    }
    public Instruction Reverse() => new Rotate(l);
}

record struct Rotate2(IReadOnlyDictionary<int,int> forward, IReadOnlyDictionary<int,int> reverse, char l): Instruction
{
    public StringBuilder Apply(StringBuilder input) 
    {
        int index = -1;
        for (int i = 0; i < input.Length; i++)
        {
            if (input[i] == l)
            {
                index = i;
                break;
            }
        }
        var r = forward[index];
        return new RotateR(r).Apply(input);
    }
    public Instruction Reverse() => new Rotate2L(reverse, forward, l);
}

record struct Rotate2L(IReadOnlyDictionary<int,int> forward, IReadOnlyDictionary<int,int> reverse, char l): Instruction
{
    public StringBuilder Apply(StringBuilder input) 
    {
        int index = -1;
        for (int i = 0; i < input.Length; i++)
        {
            if (input[i] == l)
            {
                index = i;
                break;
            }
        }
        var r = forward[index];
        return new RotateL(r).Apply(input);
    }
    public Instruction Reverse() => new Rotate2(reverse, forward, l);
}






The code below parses the input, transforming every line to the corresponding instruction.

In [172]:
IReadOnlyDictionary<int,int> forward;
IReadOnlyDictionary<int,int> reverse;
var pw = "abcdefgh";
var query = from x in pw.Select((c,i) => (c,i))
        let c = x.c
        let i = x.i
        let r = i < 4 ? i + 1 : i + 2
        let t = new Rotate(c).Apply(new StringBuilder(pw)).ToString()
        let j = t.IndexOf(c)
        select (i, j, r);
forward = query.ToDictionary(x => x.i, x => x.r);
reverse = query.ToDictionary(x => x.j, x => x.r);

var q = from line in input
        let split = line.Split(' ')
        let item = (split[0], split[1]) switch
        {
            ("move", "position") => new Move(int.Parse(split[2]), int.Parse(split[5])) as Instruction,
            ("swap", "position") => new SwapP(int.Parse(split[2]), int.Parse(split[5])),
            ("swap", "letter") => new SwapL(split[2][0], split[5][0]),
            ("reverse", "positions") => new Reverse(int.Parse(split[2]), int.Parse(split[4])),
            ("rotate", "left") => new RotateL(int.Parse(split[2])),
            ("rotate", "right") => new RotateR(int.Parse(split[2])),
            ("rotate", "based") => new Rotate2(forward, reverse, split[6][0])
        }
        select item;



Finally, loop over all instructions and scramble the password. 

In [176]:
var sb = new StringBuilder(pw);
foreach (var item in q)
    sb = item.Apply(sb);
Console.WriteLine(sb);

foreach (var item in q)
{
    var sb2 = new StringBuilder(pw);
    sb2 = item.Apply(sb2);
    sb2 = item.Reverse().Apply(sb2);
    if (sb2.ToString() != pw)
    {
        Console.WriteLine($"{pw} -> {item} -> {sb2}");
    }
}

Console.WriteLine(sb);
// "fbgdceah"
var sb3 = new StringBuilder("fbgdceah");
foreach (var item in q.Reverse())
{
    sb3 = item.Reverse().Apply(sb3);
}
Console.WriteLine(sb3);


dgfaehcb
dgfaehcb
fdhgacbe
